In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

In [2]:
spark = SparkSession\
        .builder\
        .appName("StructuredKafkaJSON")\
        .getOrCreate()

In [3]:
# Define schema of json
schema = StructType() \
        .add("nombre", StringType()) \
        .add("edad", IntegerType()) \
        .add("peso", FloatType()) \
        .add("direccion", StringType())

In [7]:
# Create DataSet representing the stream of input lines from kafka
lines = spark\
        .readStream\
        .format("kafka")\
        .option("kafka.bootstrap.servers", "localhost:9092")\
        .option("subscribe", "json_topic")\
        .load()\
        .selectExpr("CAST(value AS STRING)")\
        .select(from_json(col("value"), schema).alias("parsed_value"))\
        .select("parsed_value.*")

In [ ]:
query = lines.writeStream \
        .outputMode("append") \
        .format("console") \
        .start()

query.awaitTermination()